In [ ]:
#向量检索构建
import pandas as pd
import faiss
import numpy as np
import pickle
from tqdm import tqdm
from openai import OpenAI

# 读取 CSV 数据
df = pd.read_csv("data.csv", header=None, names=["id", "text"])

# 过滤文本长度小于5的行
df = df[df["text"].str.len() >= 10].reset_index(drop=True)

# 随机抽样3000条，不放回
df_sampled = df.sample(n=3000, random_state=42).reset_index(drop=False)

# 初始化 OpenAI 客户端
client = OpenAI(
    api_key="******",  
    base_url="https://integrate.api.nvidia.com/v1"
)

# 生成文本嵌入的函数
def get_embedding(text: str):
    try:
        response = client.embeddings.create(
            input=[text],
            model="nvidia/nv-embedcode-7b-v1",
            encoding_format="float",
            extra_body={"input_type": "passage", "truncate": "NONE"}
        )
        return response.data[0].embedding
    except Exception as e:
        print(f"嵌入失败，跳过文本：{text[:30]}... 错误：{e}")
        return None

# 添加进度条提示
print("正在生成嵌入向量，请稍候...")

# 生成嵌入并过滤失败项
texts, embeddings = [], []
for text in tqdm(df_sampled["text"], desc="Embedding"):
    emb = get_embedding(text)
    if emb is not None:
        texts.append(text)
        embeddings.append(emb)

# 转为 numpy 数组
embeddings_np = np.array(embeddings).astype("float32")

# 获取向量维度
embedding_dim = embeddings_np.shape[1]

# 构建 FAISS 索引
index = faiss.IndexFlatL2(embedding_dim)
index.add(embeddings_np)

# 保存向量索引到磁盘
faiss.write_index(index, "vector_index.faiss")

# 保存文本 ID 映射（用 pickle 序列化）
with open("id_map.pkl", "wb") as f:
    pickle.dump(texts, f)

print("向量索引与 ID 映射已成功保存。")

正在生成嵌入向量，请稍候...


Embedding: 100%|██████████| 3000/3000 [25:46<00:00,  1.94it/s]


向量索引与 ID 映射已成功保存。


In [ ]:
#测试
import faiss
import pickle
import numpy as np
from openai import OpenAI

# 加载 FAISS 索引
index = faiss.read_index("vector_index.faiss")

# 加载 ID 映射
with open("id_map.pkl", "rb") as f:
    id_map = pickle.load(f)

# 初始化客户端
client = OpenAI(
    api_key="******",
    base_url="https://integrate.api.nvidia.com/v1"
)

# 查询函数
def get_query_embedding(text: str):
    response = client.embeddings.create(
        input=[text],
        model="nvidia/nv-embedcode-7b-v1",
        encoding_format="float",
        extra_body={"input_type": "query", "truncate": "NONE"}
    )
    return response.data[0].embedding

def query(text_query: str, top_k: int = 3):
    query_vector = np.array(get_query_embedding(text_query)).astype("float32").reshape(1, -1)
    distances, indices = index.search(query_vector, top_k)
    results = [(id_map[i], distances[0][j]) for j, i in enumerate(indices[0])]
    return results

# 示例查询
results = query("有没有买西服的顾客？")
for r in results:
    print("相似文本：", r[0])
    print("相似度距离：", r[1])

相似文本： 4月25号，一位男士顾客来买面试穿的西服我们给他推荐了一款他试穿了一下觉得很好就直接买单了3424元。
相似度距离： 0.67480475
相似文本： 12月7号，一位36岁左右的先生，有券刚好需要一套西服，看上了一套蓝色暗格纹抗皱西服。
相似度距离： 0.69302225
相似文本： 4月22号，先生给自己特殊体型买一套西服。
相似度距离： 0.6937717
